In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Here I will finetune MT models and then evaluate quality of translation with BLEU and METEOR

# Dependencies

In [ ]:
! pip install transformers
! pip install sentencepiece

In [ ]:
!git clone https://github.com/google/sentencepiece.git 
%cd sentencepiece
!mkdir build

In [ ]:
%cd build
!cmake ..
!make
!make install
!ldconfig -v

In [ ]:
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm_notebook
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Loading and splitting data...

In [ ]:
# ! gdown https://drive.google.com/uc?id=1--flbLLXbMC0_pPnmPAem9pnYwIhOjjF
# ! gunzip -c /content/paracrawl-release1.en-ru.zipporah0-dedup-clean.tgz | tar xvf -
# with open('/content/paracrawl-release1.en-ru.zipporah0-dedup-clean.en') as f:
#     eng_lines = [i.strip() for i in f.readlines()]

# with open('/content/paracrawl-release1.en-ru.zipporah0-dedup-clean.ru') as f:
#     ru_lines = [i.strip() for i in f.readlines()]

YANDEX data

In [ ]:
%cd /content

In [ ]:
! gdown https://drive.google.com/uc?id=1-6VfQwe6I3t8utkGkqDiyuyrqc35Xg6c
! unzip /content/1mcorpus.zip
# ! unzip /content/drive/MyDrive/MT_sentence_simpl/1mcorpus.zip

In [ ]:
with open('/content/corpus.en_ru.1m.en') as f:
    eng_lines = [i.strip() for i in f.readlines()]

with open('/content/corpus.en_ru.1m.ru') as f:
    ru_lines = [i.strip() for i in f.readlines()]

In [ ]:
data_test = pd.DataFrame(list(zip(eng_lines, ru_lines)), columns=['src', 'dst'])
# data_test[30000:33000].to_csv('/content/drive/MyDrive/MT_sentence_simpl/translation_test_new_sample.csv', index=False)
# data_test_new.to_csv('/content/drive/MyDrive/MT_sentence_simpl/translation_test_new.csv', index=False)

In [ ]:
ru_train, ru_val, eng_train, eng_val = train_test_split(ru_lines[:30000], eng_lines[:30000], test_size=.1)
# ru_test, eng_test = ru_train[30000:33000], eng_train[30000:33000]
# test_data = pd.DataFrame(list(zip(eng_test, ru_test)), columns = ['src', 'dst'])
# test_data.to_csv('/content/drive/MyDrive/MT_sentence_simpl/translation_test.csv', index=False)
# ru_train, ru_val, eng_train, eng_val = ru_train[:30000], ru_val[:5000],\
#                                         eng_train[:30000], eng_val[:5000]

In [ ]:
! gdown https://drive.google.com/uc?id=1-paJt-Q077ZoQ3l_kDQGTbusRkun70PR
test_data = pd.read_csv('/content/data_test_sample_google.csv')
test_data.google_API = test_data.google_API.apply(lambda x: x.replace("&quot;", " "))

In [ ]:
# test_data = pd.read_csv('/content/drive/MyDrive/MT_sentence_simpl/data_test_sample_google.csv')

# Finetuning Mariam MT from Hugging Face

# Freeze encoder weights

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
model.to(device)

for param in model.base_model.parameters():
    param.requires_grad = True #False

In [ ]:
train_encodings = tokenizer.prepare_seq2seq_batch(eng_train, 
                                                  ru_train, 
                                                  truncation=True, 
                                                  padding=True,
                                                  max_length=100)
val_encodings = tokenizer.prepare_seq2seq_batch(eng_val, 
                                                ru_val, 
                                                truncation=True, 
                                                padding=True,
                                                max_length=100)

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/MT_sentence_simpl/models/marian_model_8_epochs_30k_samples_no_max_length')

In [ ]:
class Seq2seqDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["labels"])

In [ ]:
train_dataset = Seq2seqDataset(train_encodings)
eval_dataset = Seq2seqDataset(val_encodings)
model.train()

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)#64
optim = AdamW(model.lm_head.parameters(), lr=5e-5)

In [ ]:
for epoch in range(8):
    epoch_loss = []
    for batch in tqdm(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
#         embeddings = model.base_model(input_ids,\
#                                       decoder_input_ids=labels,\
#                                       attention_mask=attention_mask)/
#                                       .requires_grad(True)
#         outputs = model.lm_head(embeddings)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        epoch_loss.append(loss.item())
        loss.backward()
        optim.step()
        
    print(f"Epoch {epoch} finished; Loss : {np.mean(epoch_loss)}")

model.eval()

In [ ]:
import os
! mkdir /content/drive/MyDrive/MT_sentence_simpl/models
experiment_name = "marian_model_8_epochs_30k_samples_no_max_length"
model.save_pretrained(os.path.join("/content/drive/MyDrive/MT_sentence_simpl/models", experiment_name))

# MARIAM MT Evaluation

In [ ]:
# helper functions

def batch_generator(
        list_of_sentences,
        size=32
):
    num_batch = len(list_of_sentences)//size
    for index in range(num_batch):
        yield list_of_sentences[index*size:(index+1)*size]
    yield list_of_sentences[num_batch*size:]


def translate_to_russin(model, tok, src):

  translations_src = []
  for i in tqdm_notebook(batch_generator(src)):
    l = tok(i, return_tensors="pt", padding=True, max_length=512, truncation=True).to(device)
    translated = model.generate(**l)
    translations_src.extend([tok.decode(t, skip_special_tokens=True) for t in translated])

  return translations_src


## Original version

In [ ]:
# import en-ru model from transformers
model_name = 'Helsinki-NLP/opus-mt-en-ru'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model_name = 'Helsinki-NLP/opus-mt-en-ru'
model = MarianMTModel.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
model.to(device)

In [ ]:
# test--------------------------------------------------
sys  = translate_to_russin(model, tokenizer, list(test_data.src.values))
for i, obj  in enumerate(sys):
  sys[i] = sys[i].replace('&gt;', '')
  #src[i] = re.sub(r'[&gt;{1,}]', " ", src[i])
  sys[i] = sys[i].replace('&lt;', '')
  sys[i] = sys[i].replace('()', '')
  sys[i] = sys[i].replace('&quot;', '')
test_data['mariam_sys'] = sys
test_data.to_csv('/content/drive/MyDrive/MT_sentence_simpl/data_test_sample_google.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


## Tuned version

In [ ]:
test_data.sample(4)

,src,dst,google_API,mariam_sys
2833,You should ignore them.,Вам следует пренебрегать ими.,Вы должны игнорировать их.,Ты должен игнорировать их.
1821,It seems to me there are three candidates.,"Мне кажется, уже есть три 'кандидата'.","Мне кажется, есть три кандидата.","Мне кажется, что есть три кандидата."
361,Foreign workers account for barely one third o...,Едва лишь одну треть от общей численности труд...,Иностранные работники составляют едва ли треть...,Иностранные рабочие составляют лишь одну треть...
1142,It was no coincidence that the Kimberley Proce...,"Не было случайностью, что Кимберлийский процес...",Кимберлийский процесс зародился в Южной Африке...,Не случайно Кимберлийский процесс родился в Юж...


In [ ]:
# import en-ru model from transformers
model_name = 'Helsinki-NLP/opus-mt-en-ru'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model_name = '/content/drive/MyDrive/MT_sentence_simpl/models/marian_model_8_epochs_30k_samples_no_max_length'
model = MarianMTModel.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(62518, 512, padding_idx=62517)

In [ ]:
sys  = translate_to_russin(model, tokenizer, list(test_data.src.values))
for i, obj  in enumerate(sys):
  sys[i] = sys[i].replace('&gt;', '')
  #src[i] = re.sub(r'[&gt;{1,}]', " ", src[i])
  sys[i] = sys[i].replace('&lt;', '')
  sys[i] = sys[i].replace('()', '')
  sys[i] = sys[i].replace('&quot;', '')
test_data['mariam_tuned_sys'] = sys
test_data.to_csv('/content/drive/MyDrive/MT_sentence_simpl/data_test_sample_google.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


In [ ]:
test_data.sample(4)

,src,dst,google_API,mariam_sys,mariam_tuned_sys
522,"When these application are started via Gpass, ...",Когда это приложение запускается через TGPasst...,"Когда эти приложения запускаются через Gpass, ...","Когда это приложение запускается через Gpass, ...","Когда приложение запускается через Gpass, они ..."
823,5th International Conference on Advanced Techn...,и разработкам в целлюлозно-бумажной промышленн...,5-я Международная конференция по передовым тех...,Пятая Международная конференция по передовым т...,Пятая международная конференция по современным...
2982,(While provincial reconstruction teams are led...,(В то время как провинциальные команды реконст...,(Хотя провинциальные группы по восстановлению ...,(В то время как провинциальные группы по восст...,(Хотя провинциальные группы по восстановлению ...
532,This is all having a direct negative impact on...,Все это оказывает негативное воздействие на по...,Все это оказывает прямое негативное воздействи...,Все это оказывает непосредственное негативное ...,Все это оказывает непосредственное негативное ...


# Evaluation

In [ ]:
! pip3 install sacrebleu
! pip install -U nltk
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('wordnet')

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/MT_sentence_simpl/data_test_sample_google.csv')

### MARIAM

### BLEU

In [ ]:
import sacrebleu
refs = [list(test_data['dst'].values)]
sys = list(test_data['mariam_sys'].values)
bleu = sacrebleu.corpus_bleu(sys, refs)
print(bleu.score)

21.934412821022754


In [ ]:
refs = [list(test_data['dst'].values)]
sys = list(test_data['mariam_sys'].values)
np.mean([sacrebleu.sentence_bleu(i,j).score for i, j in list(zip(sys, refs))])

28.78787818101128

In [ ]:
# refs = []
# for i, j in data_dict.items():
#   refs.append(j)

nltk.translate.bleu_score.corpus_bleu(refs, sys)

0.16893612092623872

##### NIST

In [ ]:
from nltk.translate.nist_score import corpus_nist
refs = [[i.split()] for i in test_data['dst'].values]
sys = [i.split() for i in test_data['mariam_sys'].values]
corpus_nist(refs,sys)

5.630190713636598

#### METEOR

In [ ]:
meteor_mt = sum(meteor_score(row['dst'], row['mariam_sys']) for i, row in test_data.iterrows())/test_data.shape[0]

In [ ]:
meteor_mt

0.1541136436844775

### MARIAM Tuned

### BLEU

In [ ]:
import sacrebleu
refs = [list(test_data['dst'].values)]
sys = list(test_data['mariam_tuned_sys'].values)
bleu = sacrebleu.corpus_bleu(sys, refs)
print(bleu.score)

21.411719265727832


In [ ]:
refs = [list(test_data['dst'].values)]
sys = list(test_data['mariam_tuned_sys'].values)
np.mean([sacrebleu.sentence_bleu(i,j).score for i, j in list(zip(sys, refs))])

46.825687910244035

In [ ]:
nltk.translate.bleu_score.corpus_bleu(refs, sys)

0.16258154057153446

##### NIST

In [ ]:
from nltk.translate.nist_score import corpus_nist
refs = [[i.split()] for i in test_data['dst'].values]
sys = [i.split() for i in test_data['mariam_tuned_sys'].values]
corpus_nist(refs,sys)

5.543044277381849

##### METEOR

In [ ]:
meteor_mt = sum(meteor_score(row['dst'], row['mariam_tuned_sys']) for i, row in test_data.iterrows())/test_data.shape[0]

In [ ]:
meteor_mt

0.15623524899165783

### GOOGLE API

##### BLEU

In [ ]:
import sacrebleu
refs = [list(test_data['dst'].values)]
sys = list(test_data['google_API'].values)
bleu = sacrebleu.corpus_bleu(sys, refs)
print(bleu.score)

26.129453989406834


In [ ]:
refs = [list(test_data['dst'].values)]
sys = list(test_data['google_API'].values)
np.mean([sacrebleu.sentence_bleu(i,j).score for i, j in list(zip(sys, refs))])

54.45178846139407

In [ ]:
nltk.translate.bleu_score.corpus_bleu(refs, sys)

0.20723400512274096

##### NIST

In [ ]:
from nltk.translate.nist_score import corpus_nist
refs = [[i.split()] for i in test_data['dst'].values]
sys = [i.split() for i in test_data['google_API'].values]
corpus_nist(refs,sys)

6.405032515651133

##### METEOR

In [ ]:
meteor_mt = sum(meteor_score(row['dst'], row['google_API']) for i, row in test_data.iterrows())/test_data.shape[0]

In [ ]:
meteor_mt

0.15040614041259198

# Fairseq

In [ ]:
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
! wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz
! tar -xzvf /content/mbart.cc25.v2.tar.gz
! apt-get install cmake build-essential pkg-config libgoogle-perftools-dev

In [ ]:
!git clone https://github.com/pytorch/fairseq
%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

# Process data

In [ ]:
! rm -r /content/fairseq/data
! mkdir /content/fairseq/data

In [ ]:
ru_train, ru_val, eng_train, eng_val 
data_train = pd.DataFrame(list(zip(eng_train, ru_train)), columns=['src', 'dst'])
data_dev = pd.DataFrame(list(zip(eng_val, ru_val)), columns=['src', 'dst'])

In [ ]:
### process WikiLarge

with open('/content/fairseq/data/test.en', "a") as f:
  for i, row in test_data.iterrows():
    f.write(row['src']+'\n')

with open('/content/fairseq/data/train.en', "a") as f:
  for i, row in data_train.iterrows():
    f.write(row['src']+'\n')

with open('/content/fairseq/data/dev.en', "a") as f:
  for i, row in data_dev.iterrows():
    f.write(row['src']+'\n')

with open('/content/fairseq/data/test.ru', "a") as f:
  for i, row in test_data.iterrows():
    f.write(row['dst']+'\n')

with open('/content/fairseq/data/train.ru', "a") as f:
  for i, row in data_train.iterrows():
    f.write(row['dst']+'\n')

with open('/content/fairseq/data/dev.ru', "a") as f:
  for i, row in data_dev.iterrows():
    f.write(row['dst']+'\n')

In [ ]:
SPM="/content/sentencepiece/build/src/spm_encode"
BPE_MODEL="/content/mbart.cc25.v2/sentence.bpe.model"
DATA_DIR="/content/fairseq/data"
SRC="en"
TGT="ru" #en

!$SPM --model=$BPE_MODEL < $DATA_DIR/train.$SRC > $DATA_DIR/train.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/train.$TGT > $DATA_DIR/train.spm.$TGT &
!$SPM --model=$BPE_MODEL < $DATA_DIR/dev.$SRC > $DATA_DIR/dev.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/dev.$TGT > $DATA_DIR/dev.spm.$TGT &
!$SPM --model=$BPE_MODEL < $DATA_DIR/test.$SRC > $DATA_DIR/test.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/test.$TGT > $DATA_DIR/test.spm.$TGT &

In [ ]:
PREPROCESSED_DATA_DIR="/content/fairseq/data"
DICT="/content/mbart.cc25.v2/dict.txt"
!fairseq-preprocess \
  --source-lang en \
  --target-lang ru \
  --trainpref /content/fairseq/data/train.spm \
  --validpref /content/fairseq/data/dev.spm \
  --testpref /content/fairseq/data/test.spm \
  --destdir /content/fairseq/data \
  --thresholdtgt 0 \
  --thresholdsrc 0 \
  --srcdict /content/mbart.cc25.v2/dict.txt \
  --tgtdict /content/mbart.cc25.v2/dict.txt \
  --workers 70

2021-04-29 00:15:29 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/content/fairseq/data', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, simul_type=None, source_lang='en', srcdict='/content/mbart.cc25.v2/dict.txt', suppress_crashes=False, target_lang='ru', task='translation', tensorboard_logdir=None, testpref='/conte

# Training

Also, it is necessary to make the following change in /content/fairseq/fairseq/tasks/translation_from_pretrained_bart.py:

```
def __init__(self, args, src_dict, tgt_dict):
        super().__init__(args, src_dict, tgt_dict)
        self.args = args                  # add this line !!!!!
        self.langs = args.langs.split(",")
        for d in [src_dict, tgt_dict]:
            for l in self.langs:
```

In [ ]:
# mkdir to put the checkpoints
! mkdir /content/drive/MyDrive/checkpoints_mt

In [ ]:
!fairseq-train /content/fairseq/data \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 54725  \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 5 \
  --source-lang en --target-lang ru \
  --batch-size 16 \
  --memory-efficient-fp16 \
  --validate-interval 1 \
  --patience 3 \
  --max-epoch 5 \
  --save-interval 5 --keep-last-epochs 10 --keep-best-checkpoints 2 \
  --ddp-backend no_c10d \
  --seed 42 --log-format simple --log-interval 500 \
  --restore-file /content/drive/MyDrive/checkpoints_mt/checkpoint_last.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN \
  --scoring bleu \
  --save-dir /content/drive/MyDrive/checkpoints_mt

2021-04-29 01:09:32 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 500, 'log_format': 'simple', 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 42, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_w

In [ ]:
!fairseq-generate /content/fairseq/data \
  --path  /content/drive/MyDrive/checkpoints_mt/checkpoint_best.pt \
  --task translation_from_pretrained_bart \
  --gen-subset test \
  --source-lang en --target-lang ru \
  --bpe 'sentencepiece' --sentencepiece-model /content/mbart.cc25.v2/sentence.bpe.model \
  --sacrebleu --remove-bpe 'sentencepiece' \
  --batch-size 32 --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN > model_prediction.txt & 
!cat model_prediction.txt | grep -P "^H" |sort -V |cut -f 3- > model_prediction_mt.hyp


In [ ]:
!cp /content/model_prediction_en_ru_ru_filtered.hyp /content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru_ru_filtered.hyp


In [ ]:
with open('/content/model_prediction_mt.hyp') as f:
  lines = [i.strip() for i in f.readlines()]


test_data['fairseq_mbart'] = lines
test_data.to_csv('/content/drive/MyDrive/MT_sentence_simpl/data_test_sample_google.csv', index=False)

# Fairseq mBART

##### BLEU

In [ ]:
import sacrebleu
refs = [list(test_data['dst'].values)]
sys = list(test_data['fairseq_mbart'].values)
bleu = sacrebleu.corpus_bleu(sys, refs)
print(bleu.score)

12.315905780266137


In [ ]:
refs = [list(test_data['dst'].values)]
sys = list(test_data['fairseq_mbart'].values)
np.mean([sacrebleu.sentence_bleu(i,j).score for i, j in list(zip(sys, refs))])

34.35599238920265

In [ ]:
# from nltk.translate.bleu_score import corpus_bleu
# refs = [[i.split()] for i in test_data['dst'].values]
# sys = [i.split() for i in test_data['fairseq_mbart'].values]
# corpus_bleu(refs,sys)

In [ ]:
nltk.translate.bleu_score.corpus_bleu(refs, sys)

0.0832559310080964

##### NIST

In [ ]:
from nltk.translate.nist_score import corpus_nist
refs = [[i.split()] for i in data_test['dst'].values]
sys = [i.split() for i in data_test['fairseq_mbart'].values]
corpus_nist(refs,sys)

5.6946908482302644e-05

#### METEOR

In [ ]:
! pip3 install sacrebleu
! pip install -U nltk
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('wordnet')

Requirement already up-to-date: nltk in /usr/local/lib/python3.7/dist-packages (3.6.2)
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
meteor_mt = sum(meteor_score(row['dst'], row['google_API']) for i, row in test_data.iterrows())/test_data.shape[0]
meteor_mt

0.15064013355701317

# Ultimate translation set and Cos Sim Evaluation

In [ ]:
! pip install googletrans
! pip install transformers
! pip install laserembeddings
! pip install sentence_transformers
! pip install language_tool_python
! pip install textstat

In [ ]:
! python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
import pandas as pd
import numpy as np
import torch
import language_tool_python
import re
import textstat
from laserembeddings import Laser
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import MarianMTModel, MarianTokenizer
from transformers.hf_api import HfApi
from googletrans import Translator
from tqdm import tqdm_notebook

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/MT_sentence_simpl/data_test_sample_google.csv')
data.head()

,Unnamed: 0,src,dst,google_API,mariam_sys,mariam_tuned_sys,fairseq_mbart
0,0,"And there is no need to generalize, good relations with Armenians from abroad are being built here as well.","И не надо обобщать, здесь тоже по-разному строятся отношения с армянами из-за рубежа.","И не надо обобщать, здесь также строятся хорошие отношения с армянами из-за границы.","И нет необходимости обобщать, здесь также строятся хорошие отношения с армянами из-за границы.","И нет необходимости обобщать, здесь также строятся хорошие отношения с армянами из-за рубежа.","И не нужно говорить о том, что хорошие отношения с армянами из-за внешнего мира будут построены здесь."
1,1,"Silva, YSU: I myself am from Javakheti and I live among Armenians, the communication there is only in Armenian and schools are also in Armenian.","Сильва, студентка ЕГУ, будущий фармацевт: Я сама из Джавахети и живу там среди армян и общение там только на армянском и школы армянские.","Сильва, ЕГУ: Я сам из Джавахети и живу среди армян, общение там только на армянском, школы тоже на армянском.","Сильва, ЙСУ (говорит по-английски): Я сам из Явахети и я живу среди армян, связь существует только на армянском языке, а школы также на армянском языке.","Silva, YSU: Я сам из Javakheti и я живу среди армян, коммуникация есть только в армянском, и школы тоже на армянском.","Silva, YSU: Я сам из Javakheti и живу среди Армянцев, коммуникации есть только в Армяне и школы также в Армяне."
2,2,Laura Khanamiryan: I want to say that regretfully the only well preserved Armenian churches are in Iran.,"Лаура Ханамирян: Хочу сказать, что, к сожалению, самыми сохраненными являются армянские церкви в Иране.","Лаура Ханамирян: Я хочу сказать, что, к сожалению, единственные хорошо сохранившиеся армянские церкви находятся в Иране.","Лора Ханамирян (говорит по-английски): Я хочу сказать, что, к сожалению, единственные хорошо сохранившиеся армянские церкви находятся в Иране.","Лора Ханамирян: к сожалению, единственные хорошо сохранившиеся армянские церкви находятся в Иране.","Laura Khanamiryan: Я хочу сказать, что единственные хорошо сохраненные армийские церкви в Иране."
3,3,"Packages are the different groups of files that make up a particular piece of software, along with the instructions Ubuntu needs to install and remove the software, dependency information, and so on.","В состав пакетов входят различные файлы, которые представляют собой конкретные части программного обеспечения, а также инструкции для Ubuntu, необходимые для установки и удаления программного обеспечения, информацию о зависимостях, и так далее.","Пакеты - это различные группы файлов, которые составляют определенную часть программного обеспечения, а также инструкции, необходимые Ubuntu для установки и удаления программного обеспечения, информацию о зависимостях и т. Д.","Пакеты представляют собой различные группы файлов, которые составляют конкретный элемент программного обеспечения, вместе с инструкциями Ubuntu, необходимыми для установки и удаления программного обеспечения, информации о зависимости и т.д.","Пакеты представляют собой различные группы файлов, составляющих определенный элемент программного обеспечения, наряду с инструкциями Убунту, необходимыми для установки и удаления программного обеспечения, информации о зависимости и т.д.","Packages являются различными группами файлов, которые составляют определенную часть программного обеспечения, вместе с указаниями Ubuntu необходимо установить и удалить программное обеспечение, зависимость и так далее."
4,4,Ubuntu is based on the Debian Linux distribution but contains only a subset of Debian's software due to the amount of testing required to support each package. universe is a subcategory that contains software that the community has packaged and supports for Ubuntu.,"Система Ubuntu базируется на дистрибутиве Debian Linux, но в ней содержится только часть программного обеспечения Debian из-за того, что для поддержки каждого пакета требуется выполнение большого количества тес

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SentenceTransformer('LaBSE')
model.to(device)

def calc_cos_sim_(df, model, src, dst, column_name):
    LABSE = []
   
    for index, row in df.iterrows():

        # original
          emb_source = model.encode(row[src])
          emb_target = model.encode(row[dst])

          cos_val = cosine_similarity(emb_source.reshape(-1, emb_source.shape[0]), emb_target.reshape(-1, emb_target.shape[0]))[0][0]
          LABSE.append(cos_val)

    df[column_name] = LABSE


calc_cos_sim_(data, model, 'src', 'google_API', 'cos_sim_google')
calc_cos_sim_(data, model, 'src', 'mariam_sys', 'cos_sim_mariam')
calc_cos_sim_(data, model, 'src', 'mariam_tuned_sys', 'cos_sim_mariam_tuned')
calc_cos_sim_(data, model, 'src', 'fairseq_mbart', 'cos_sim_mbart')

In [ ]:
data.cos_sim_google.mean(), data.cos_sim_mariam.mean(), data.cos_sim_mariam_tuned.mean(), data.cos_sim_mbart.mean()

(0.8811880518396695, 0.873996279189984, 0.8745776765048504, 0.8442612900535266)